# Panels and Widgets

In [1]:
import anyio
import ipylab
import ipywidgets as ipw

app = await ipylab.App().ready()

## Panel

A `Panel` widget is the same as a `ipywidget.Box`, but with a `Title` that is used when the panel is added to the application shell.

In [2]:
panel = ipylab.Panel(children=[ipw.Dropdown()])

To quickly add the panel to the JupyterLab *shell* main area:

In [3]:
sc = await panel.add_to_shell(mode=ipylab.InsertMode.split_right, activate=False)

In [4]:
sc in panel.connections

True

In [5]:
sc in app.shell.connections

True

`sc` is a `ShellConnection`. 
It provides an `activate` method, and in further may have other features added.

In [6]:
await sc.activate()  # Will activate before returning to this notebook

Closing the connection will remove the panel from the shell but leave the panel open.

In [7]:
sc.close()

We can put it back in the shell.

In [8]:
sc = await panel.add_to_shell(mode=ipylab.InsertMode.split_right, activate=False)

In [9]:
# closable is on the widget in the shell rather than the panel, but we can set it using set_property.
await sc.set_property("title.closable", False)

The title label can be updated as required.

In [10]:
panel.title.label = "This panel has a dropdown"

We can close the panel and the view will disappear.

In [11]:
panel.close()

When the panel is closed sc is also closed.

In [12]:
sc

ShellConnection('ipylab-ShellConnection|0c4a05d8-5955-4e32-af6b-d0f880306a54')

In the case of sliders and other widgets that fit on a single line, they can even be added to the top area directly:

In [13]:
slider = ipw.IntSlider()
await app.shell.add(slider, area=ipylab.Area.top)

ShellConnection('ipylab-ShellConnection|36bf5463-6e33-4f38-ac79-24038325c21d')

We can also remove it from the top area when we are done.

In [14]:
slider.close()

## SplitPanel
A split panel is a subclass of Panel that provides a draggable border between widgets, whose orientatation can be either horizontal or vertical.
Let's create a `SplitPanel` with a few widgets inside.

In [15]:
split_panel = ipylab.SplitPanel()
progress = ipw.IntProgress(
    value=7,
    min=0,
    max=100,
    step=1,
    description="Loading:",
    bar_style="info",
    orientation="horizontal",
    layout={"height": "30px"},
)
slider_ctrl = ipw.IntSlider(
    min=0,
    max=100,
    step=1,
    description="Slider Control:",
)

# link the slider to the progress bar
ipw.jslink((slider_ctrl, "value"), (progress, "value"))

# add the widgets to the split panel
split_panel.children = [progress, slider_ctrl]
ipw.Box(children=[split_panel], layout={"height": "100px"})

Box(children=(SplitPanel(children=(IntProgress(value=7, bar_style='info', description='Loading:', layout=Layou…

In [16]:
split_panel.title.label = "A SplitPanel "
split_panel.title.icon_class = "jp-PythonIcon"

> As an alternative to `icon_class`, a `Panel` can also use custom [icons](./icons.ipynb).

In [17]:
await split_panel.add_to_shell(area=ipylab.Area.main, mode=ipylab.InsertMode.split_bottom)

ShellConnection('ipylab-ShellConnection|f5beffcc-720e-44af-a0b0-4d1d26cae4fb')

The orientation can be updated on the fly:

In [18]:
split_panel.orientation = "horizontal"

Let's put it back to `vertical`

In [19]:
split_panel.orientation = "vertical"

Just like with boxes, we can add an existing widget (the progress bar) more than once:

In [20]:
split_panel.children += (progress,)

Or add a new widget:

In [21]:
play = ipw.Play(min=0, max=100, step=1, description="Press play")
ipw.jslink((play, "value"), (slider_ctrl, "value"))
split_panel.children += (play,)

## Left and Right Areas

The same `SplitPanel` widget (or `Panel` or `Widget`) can be moved to the left area:

In [22]:
sc = await split_panel.add_to_shell(area=ipylab.Area.left, rank=1000)
await sc.activate()

Or to the right area:

In [23]:
sc = await split_panel.add_to_shell(area=ipylab.Area.right, rank=1000)
await sc.activate()

In [24]:
await app.shell.collapse_right()

Notice how it moved the widget instead of adding a second one?

This is the default behaviour.

To have multiple widgets, provide it with a new `connection_id` when 'adding' it to the shell.

In [25]:
sc = await split_panel.add_to_shell(connection_id=ipylab.ShellConnection.to_id(), mode=ipylab.InsertMode.split_right)

In [26]:
await sc.activate()
split_panel.connections

(ShellConnection('ipylab-ShellConnection|f5beffcc-720e-44af-a0b0-4d1d26cae4fb'),
 ShellConnection('ipylab-ShellConnection|2ef52bdf-3612-470b-9ab6-9a475dd7f8a0'))

In [27]:
split_panel.close()
await anyio.sleep(0.1)

In [28]:
split_panel.connections

(< CLOSED: ShellConnection('ipylab-ShellConnection|f5beffcc-720e-44af-a0b0-4d1d26cae4fb') >,
 < CLOSED: ShellConnection('ipylab-ShellConnection|2ef52bdf-3612-470b-9ab6-9a475dd7f8a0') >)